# AutoUSD, sUSDe, Aave USDC and BIL (1-3 month T-Bill ETF) Comparison

Task:


> 
> Metrics:
> APY  
> Yield Volatility (annualized)  
> Excess vs 3M T-Bill  
> Information Ratio  
> For products:  
> USD: autoUSD, Ethena sUSDe, Aave USDC lending, US Gov short term treasuries  
> ETH: autoETH, Lido staking (stETH), Aave ETH lending  
> Time periods: 30 day, 1 year, since inception  
> *no 1 year on autoUSD since <1 year old  
> Definitions:  
> Yield Volatility: stdev of monthly net returns × √12  
> Excess vs Benchmark: difference between average net APY and the benchmark (3M T-bill for USD; Lido staking rate for ETH).  
> Information Ratio: average monthly excess return ÷ stdev of monthly excess returns, annualized  


# Methods


## Data sources:

Onchain calls on mainnet at the last block of each day

- autoUSD.converToAssets(1e18) / 1e6 -> USDC value of one autoUSD share
- sUSDe -> chainlink sUSDE -> USD oracle, safe USD value of sUSDe over time.
- Aave USDC lending, stataAaveUSDC.convertToAssets(1e6) / 1e6, growth of a share of the aave USDC vault

BIL `SPDR Bloomberg 1-3 Month T-Bill ETF`

https://www.ssga.com/us/en/intermediary/etfs/spdr-bloomberg-1-3-month-t-bill-etf-bil 

> The SPDR® Bloomberg  1-3 Month T-Bill ETF seeks to provide investment results that, before fees and expenses, correspond generally to the price and yield performance of the Bloomberg  1-3 Month U.S. Treasury Bill Index (the "Index")  
> Seeks to provide exposure to publicly issued U.S. Treasury Bills that have a remaining maturities between 1 and 3 months  
> Short duration fixed income is less exposed to fluctuations in interest rates than longer duration securities  
> Rebalanced on the last business day of the month  


As of Oct 26, 2025 BIL has ~42.5B AUM

```
!pip install yfinance
import yfinance as yf
tickers = ['BIL']
data = yf.download(tickers, start="2024-10-26", end="2025-10-26", actions=True)
data.to_csv('data.csv')

```



In [8]:
from multicall import Call

import pandas as pd
import numpy as np
import plotly.express as px

from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)
from mainnet_launch.constants import ETH_CHAIN, AUTO_ETH


# https://etherscan.io/address/0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6#readProxyContract
# chainlink safe price of sUSDe over time
# https://etherscan.io/address/0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c aave USDC
# https://data.chain.link/feeds/ethereum/mainnet/susde-usd


def _process_chainlink_usd_price(success, args):
    """
    Normalize Chainlink latestRoundData -> price as float (answer / 1e6).
    args is: (roundId:uint80, answer:int256, startedAt:uint256, updatedAt:uint256, answeredInRound:uint80)
    """
    round_id, answer, started_at, updated_at, answered_in_round = args
    if success:
        return float(answer) / 1e18


stETH_ETH_chainlink_oracle = "0x86392dC19c0b719886221c78AB11eb8Cf5c52812"

chainlink_stETH_price_call = Call(
    stETH_ETH_chainlink_oracle,
    ["latestRoundData()((uint80,int256,uint256,uint256,uint80))"],
    [("stETH_ETH_safe_price", _process_chainlink_usd_price)],
)

stata_aave = "0xC077F28FC0f10C886253E67Be402A7F7D611D4EE"  # convert to shares calls


aave_convert_to_shares_call = Call(
    stata_aave,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("WETH_aave", safe_normalize_with_bool_success)],
)

autoETH_nav_per_share_call = Call(
    AUTO_ETH.autopool_eth_addr,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("autoETH", safe_normalize_with_bool_success)],
)

wstETH = "0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0"

wstETH_convert_to_shares_call = Call(
    wstETH,
    ["stEthPerToken()(uint256)"],
    [("wstETH", safe_normalize_with_bool_success)],
)


insta_dapp_stETH_looping = "0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78"

ieth_convert_to_assets_call = Call(
    insta_dapp_stETH_looping,
    ["convertToAssets(uint256)(uint256)", int(1e18)],  # 1.0 share → assets
    [("iETH_v2", safe_normalize_with_bool_success)],
)


end_of_day_oct_26_2024_block = 21053236
blocks = build_blocks_to_use(ETH_CHAIN, start_block=end_of_day_oct_26_2024_block)
df = get_raw_state_by_blocks(
    calls=[
        aave_convert_to_shares_call,
        chainlink_stETH_price_call,
        autoETH_nav_per_share_call,
        wstETH_convert_to_shares_call,
        ieth_convert_to_assets_call,
    ],
    blocks=blocks,
    chain=ETH_CHAIN,
)

df["wstETH_eth_price"] = df["wstETH"] * df["stETH_ETH_safe_price"]
df["iETH_v2_eth_price"] = df["iETH_v2"] * df["stETH_ETH_safe_price"]
df = df[["autoETH", "WETH_aave", "wstETH_eth_price", "iETH_v2_eth_price"]]
df.index = pd.to_datetime(df.index, unit="s").date
start_date = pd.to_datetime("2024-11-16").date()  # started Sep 17th, 2024
end_date = pd.to_datetime("2025-11-01").date()

end_of_day_prices = df[(df.index >= start_date) & (df.index <= end_date)].copy()
end_of_day_prices.index = pd.to_datetime(end_of_day_prices.index)
end_of_day_prices

,autoETH,WETH_aave,wstETH_eth_price,iETH_v2_eth_price
2024-11-16,1.004854,1.034775,1.184499,1.130873
2024-11-17,1.005219,1.034830,1.184630,1.131085
2024-11-18,1.005444,1.034886,1.184878,1.131338
2024-11-19,1.005521,1.034941,1.184896,1.131464
2024-11-20,1.005430,1.034995,1.184716,1.131345
...,...,...,...,...
2025-10-28,1.053894,1.054873,1.217650,1.184809
2025-10-29,1.054042,1.054924,1.217743,1.184915
2025-10-30,1.054015,1.054976,1.217837,1.185028
2025-10-31,1.054047,1.055029,1.217575,1.184776


In [9]:
from mainnet_launch.adhoc.returns_math import (
    compute_series_apy,
    compute_most_recent_30_days_apy,
    compute_yield_volatility_annualized,
    compute_excess_vs_benchmark_apy,
    compute_information_ratio,
    compute_30_day_excess_vs_benchmark_apy,
)

start_date = end_of_day_prices.index.min()
end_date = end_of_day_prices.index.max()
thrity_days_before_end = end_date - pd.Timedelta(days=30)
all_time_apy = compute_series_apy(end_of_day_prices)
most_recent_30_days_apy = compute_most_recent_30_days_apy(end_of_day_prices)
yield_volatility = compute_yield_volatility_annualized(end_of_day_prices)
excess_apy_vs_benchmark = compute_excess_vs_benchmark_apy(end_of_day_prices, "wstETH_eth_price")
most_recent_excess_vs_benchmark = compute_30_day_excess_vs_benchmark_apy(end_of_day_prices, "wstETH_eth_price")
information_ratio = compute_information_ratio(end_of_day_prices, "wstETH_eth_price")

combined_df = pd.concat(
    [
        all_time_apy.rename("All Time APY"),
        most_recent_30_days_apy.rename("30 Day APY"),
        yield_volatility.rename("Yield Volatility"),
        excess_apy_vs_benchmark.rename("Excess APY vs Benchmark"),
        most_recent_excess_vs_benchmark.rename("30 Day Excess vs Benchmark"),
        information_ratio.rename("Information Ratio"),
    ],
    axis=1,
)

combined_df = combined_df.reindex(
    [
        "wstETH_eth_price",
        "autoETH",
        "WETH_aave",
        "iETH_v2_eth_price",
    ]
)
combined_df = combined_df.round(2)

combined_df["All Time APY"] = combined_df["All Time APY"].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)
combined_df["30 Day APY"] = combined_df["30 Day APY"].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)
combined_df["Excess APY vs Benchmark"] = combined_df["Excess APY vs Benchmark"].apply(
    lambda x: str(x) + "%" if pd.notnull(x) else x
)
combined_df["30 Day Excess vs Benchmark"] = combined_df["30 Day Excess vs Benchmark"].apply(
    lambda x: str(x) + "%" if pd.notnull(x) else x
)
combined_df["Yield Volatility"] = combined_df["Yield Volatility"].apply(lambda x: str(x) + "%" if pd.notnull(x) else x)
print(f"Data from {end_of_day_prices.index.min().date()} to {end_of_day_prices.index.max().date()}")
print(combined_df.to_markdown())

Data from 2024-11-16 to 2025-11-01
|                   | All Time APY   | 30 Day APY   | Yield Volatility   | Excess APY vs Benchmark   | 30 Day Excess vs Benchmark   |   Information Ratio |
|:------------------|:---------------|:-------------|:-------------------|:--------------------------|:-----------------------------|--------------------:|
| wstETH_eth_price  | 2.92%          | 2.98%        | 0.37%              | 0.0%                      | 0.0%                         |              nan    |
| autoETH           | 5.09%          | 1.14%        | 0.76%              | 2.17%                     | -1.84%                       |                0.86 |
| WETH_aave         | 2.05%          | 1.79%        | 0.18%              | -0.87%                    | -1.19%                       |               -0.62 |
| iETH_v2_eth_price | 4.98%          | 5.2%         | 0.7%               | 2.06%                     | 2.22%                        |                1.15 |


: 